The charts below record cumulative cases by date of issue and cumulative case rates by date of issue. You can control the content of the output data via the cases and rates in the left hand widget. You can also select the output form of the curve via the 'linear' and 'log' options in the right hand widget.

Alternatively, you can click on the 'update' button to get the latest data and update the graph.
Enjoy yourself! Byebye!

In [74]:
# import the necessary moduels

In [75]:
from uk_covid19 import Cov19API

In [76]:
import json

In [77]:
import pandas as pd

In [78]:
import matplotlib.pyplot as plt

In [79]:
import IPython.display as clear_output

In [80]:
import numpy as np

In [81]:
import ipywidgets as wdg

In [82]:
# design the filter
filters = ['areaType=overview']

In [83]:
# design and name the structure
structure={
    "date":"date",
    "cases":"cumCasesByPublishDate",
    "rate":"cumCasesByPublishDateRate",
    
}

In [84]:
# access the api
api=Cov19API(filters=filters, structure=structure)

In [85]:
timerecord=api.get_json()

In [86]:
# store the data as timerecord.json
with open('timerecord.json', 'wt') as OUTF:
    json.dump(timerecord, OUTF)

In [87]:
# open the file and load the data
with open('timerecord.json', 'rt') as INFILE:
    data=json.load(INFILE)

In [88]:
# remove the header
datalist=data['data']

In [89]:
# filter out 'date' data
dates=[dictionary['date'] for dictionary in datalist]

In [90]:
dates.sort()

In [91]:
# convert to the date type under pandas
def convert_date(datestring):
    return pd.to_datetime(datestring, format='%Y-%m-%d')

In [92]:
# find out the start date and the end date
fromdate=convert_date(dates[0])
todate=convert_date(dates[-1])

In [93]:
# use the end numbers as index
ranknumber=pd.date_range(fromdate, todate, freq='D')
ranknumber

DatetimeIndex(['2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03',
               '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07',
               '2020-02-08', '2020-02-09',
               ...
               '2022-05-11', '2022-05-12', '2022-05-13', '2022-05-14',
               '2022-05-15', '2022-05-16', '2022-05-17', '2022-05-18',
               '2022-05-19', '2022-05-20'],
              dtype='datetime64[ns]', length=841, freq='D')

In [94]:
# create an empty dataframe
timerecorddf=pd.DataFrame(index=ranknumber, columns=['cases', 'rate'])

In [95]:
for element in datalist:
    date=convert_date(element['date'])
    for column in ['cases', 'rate']:
        value=float(element[column]) if element[column]!=None else 0.0
        timerecorddf.loc[date, column]=value

In [96]:
# timerecorddf.plot

In [97]:
# create SelectMultiple controls
Variables=wdg.SelectMultiple(
    options=['cases', 'rate'],
    value=['cases', 'rate'],
    rows=2,
    description='Valuables: ',
    disabled=False
)

In [98]:
# create RadioButtons controls
Type=wdg.RadioButtons(
    options=['linear','log'],
    
    discription='Type: ',
    disabled=False)

In [99]:
# horizontally place the two controls
position=wdg.HBox([Variables, Type])

In [100]:
# When users change the selection, graph update
def timerecord_graph(count, graphtype):
    if graphtype=='linear':
        logType=False
    else:
        logType=True
    n=len(count)
    if n>0:
        timerecorddf[list(count)].plot(logy=logType)
        plt.show()
    else:
         print("Click to select something")            

In [101]:
graph=wdg.interactive_output(timerecord_graph, {'count':Variables, 'graphtype':Type})

In [102]:
# dashboard part

In [103]:
# go to PHE and get the latest api data
def access_api():
    filters = ['areaType=overview']
    structure={
    "date":"date",
    'cases':'cumCasesByPublishDate',
    'rate':'cumCasesByPublishDateRate',
    
}
    api=Cov19API(filters=filters, structure=structure)
    
    timerecord=api.get_json()
    with open('timerecord.json', 'wt') as OUTF:
        json.dump(timerecord, OUTF)
    with open('timerecord.json', 'rt') as INFILE:
        data=json.load(INFILE)
#     最后得到的是新的api数据
    return data

In [104]:
# create empty dataframe and fill it with values
def wrangle_data():
    data=access_api()
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    fromdate=convert_date(dates[0])
    todate=convert_date(dates[-1])
    ranknumber=pd.date_range(fromdate, todate, freq='D')
    timerecorddf=pd.DataFrame(index=ranknumber, columns=['cases', 'rate'])
    for element in datalist:
        date=convert_date(element['date'])
        for column in ['cases','rate']:
            if pd.isna(timerecorddf.loc[date, column]):
                value=float(element[column]) if element[column]!=None else 0.0
                timerecorddf.loc[date, column]=value
    timerecorddf.fillna(0.0, inplace=True)
    return  timerecorddf

In [105]:
# A function that updates the graph
def refresh_graph():
    current=Variables.value
    if current==Variables.options:
        other=Variables.options
    else:
        other=Variables.options
    Variables.value=other
    Variables.value=current

In [106]:
# callback fuction, when users click the button, call back series of fucntions below.
def api_button_callback(button):
    global timerecorddf
    timerecorddf=wrangle_data()
    refresh_graph()
    apibutton.icon="check"
    apibutton.disabled=True

In [107]:
# create a button
apibutton=wdg.Button(
    description='update',
    disabled=False,
    button_style='success',
    tooltip='Click to update the latest data from PHE',
    icon='download'
)


In [108]:
# connect the callback function with the button
apibutton.on_click(api_button_callback)
display(apibutton)


Button(button_style='success', description='update', icon='download', style=ButtonStyle(), tooltip='Click to u…

In [109]:
# show what we make
display(position, graph)

Output()